In [ ]:
%load_ext autoreload
%autoreload 2
import chromdyn
print("chromdyn path:", chromdyn.__file__)

In [ ]:
import os
import sys
#sys.path.append('../chromdyn')
from chromdyn.topology import TopologyGenerator
from chromdyn.chromatin_dynamics import ChromatinDynamics
from chromdyn.traj_utils import save_pdb

from chromdyn.hic_utils import HiCManager
hicman = HiCManager()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
generator = TopologyGenerator()
n_beads_per_chain = 100
chain_num = 1

N = n_beads_per_chain * chain_num # total number of beads
# generate chain list
chain_list = [n_beads_per_chain for i in range(chain_num)]
type_list = []
block_length = 20
for i in range(N):
    if (i // block_length) % 2 == 0:
        type_list.append("A")
    else:
        type_list.append("B")

generator.gen_top(chain_list, type_list)

mass = 0.001
friction = 1.0

In [ ]:
# Choose available platform
sim = ChromatinDynamics(generator.topology, name = 'testPBC', platform_name = "CUDA", output_dir = "output", console_stream = True, mass = mass)

#Add forces
kb = 30.0
lb = 1.0
sim.force_field_manager.add_harmonic_bonds(r0 = lb, k = kb,)# this function will automatically add bonds according to the topology file

kbound = 30.0
# note the formula of self avoidance here is different from OpenMiChroM
sim.force_field_manager.add_self_avoidance(Ecut = 5.0, k = 20.0, r = 1.0) #r = sigma = 2*radius

In [ ]:
block_size_prod = 200 # 0.1 tau_sim
T = 0.0
sim.simulation_setup(
    init_struct='randomwalk',
    integrator='active-langevin',
    temperature=T,
    timestep=0.0005,
    friction=friction,
    save_pos=True,
    save_energy=True,
    energy_report_interval=5_000,
    pos_report_interval=block_size_prod,    
    PBC = True,
    box_vectors = (
        (10.0, 0.0, 0.0),
        (0.0, 10.0, 0.0),
        (0.0, 0.0, 10.0)
    )   
)

print("Simulating under temperature:", T)

In [ ]:
F = 1.0
tau = 1.0

type_array = np.array(type_list)
F_seq = np.zeros(N)
F_seq = np.array((type_array == 'A').astype(int))*  F
print(F_seq)
tau_seq=[tau]*sim.num_particles

sim.set_activity(F_seq=F_seq, tau_seq=tau_seq)

In [ ]:
simulation = sim.simulation
system = simulation.system
context = simulation.context
print(f"System uses PBC? {system.usesPeriodicBoundaryConditions()}")


In [ ]:
# step 3: relaxasion simulation
relaxation_steps = 100 * block_size_prod # 5000 tau_sim
sim.run(relaxation_steps, report=False)

reporter = sim.reporters['stability']
print(reporter.kinetic_threshold) # this should show 5.0
reporter.kinetic_threshold = 80.0
print(reporter.kinetic_threshold) # this should show 80.0

In [ ]:
simulation = sim.simulation
system = simulation.system
context = simulation.context
print(f"System uses PBC? {system.usesPeriodicBoundaryConditions()}")

In [ ]:

n_blocks_prod = 1000
running_steps = n_blocks_prod * block_size_prod
v_history = []

for _ in range(n_blocks_prod):
    sim.run(block_size_prod, report=True)
    state = sim.simulation.context.getState(getVelocities=True)
    current_velocities = state.getVelocities(asNumpy=True)
    v_history.append(current_velocities)

# save velocity history
v_history = np.array(v_history)
np.save(os.path.join('output', f"velocity_history.npy"), v_history)
del v_history

# verify potential forces are added
sim.print_force_info()


In [ ]:
#traj_file=sim.reporters.get('position').filename
traj_file='output/testPBC_positions.cndb'
velocity_file = os.path.join('output', f"velocity_history.npy")


In [ ]:
# tools.py rely on OpenMiChroM.CndbTools, one may not choose to use it
r'''from tools import ndb2cndb, load, xyz
from OpenMiChroM.CndbTools import cndbTools
cndb_tool = cndbTools()
traj = load(cndb_tool, filename = traj_file)'''

In [ ]:
from chromdyn.traj_utils import ChromatinTrajectory
# in this new class, we can load the file and extract the trajectory with .load(), .xyz(), 
# or one can import load_trajectory(),get_xyz() function, they're the same, with self a ChromatinTrajectory object.
traj = ChromatinTrajectory(traj_file)
xyz = traj.xyz(frames=[0, None, 1], beadSelection=None)


In [ ]:
# calculate Rg by types
from chromdyn.traj_utils import Analyzer
#Analyzer.compute_RG_type
rg_result = Analyzer.compute_RG_type(traj)
rg_result.keys()

In [ ]:
#print(xyz)

In [ ]:
# check if the box vectors are correctly saved, should be (Nframes, 3, 3)
np.shape(traj.box_vectors)

In [ ]:
traj.box_vectors

In [ ]:
# calculate PBC-HiC(image contact included)
hic = hicman.gen_pbc_hic_from_cndb(traj_file = traj_file, mu = 3.22, rc = 1.78,p = None, platform = 'CUDA', batch_size = 100)

In [ ]:
traj.ChromSeq


In [ ]:
traj.topology

In [ ]:
traj.topology.bonds

In [ ]:
traj.topology.chains

In [ ]:
traj.topology.n_chains

In [ ]:
traj.Nbeads

In [ ]:
traj.topology.chain_info

In [ ]:
import inspect

# print the source code of traj.xyz
print(inspect.getsource(traj.xyz))
print(traj.xyz.__code__.co_filename)

In [ ]:
from chromdyn.visualization import visualize, visualize_animation
# these 2 visualization function are for the new format of cndb files with pbc mode on
pbc_box_side_length = 10
visualize(
     traj = traj,
     select_frame=532,
     axis_limits=(-10, pbc_box_side_length + 10, -10, pbc_box_side_length + 10, -10, pbc_box_side_length + 10), # Optional
     colors=None, # Optional
     isring=False,
     r=0.5,
     PBC=True,
     color_mode='type'
 )



In [ ]:
from chromdyn.visualization import visualize_pbc_images
visualize_pbc_images(
    traj = traj,
    select_frame = 532,
    n_layers = 1, # 3*3*3 grid
    image_alpha = 0.15,
    image_style = 'scatter',
    r = 0.5,
    color_mode = 'type'
)

In [ ]:
plt.rcParams['animation.html'] = 'jshtml'
pbc_box_side_length = 10
visualize_animation(
     traj = traj,
     #box_a=pbc_box_side_length,
     start_frame = 0,
     end_frame = 300,
     axis_limits=(-10, pbc_box_side_length + 10, -10, pbc_box_side_length + 10, -10, pbc_box_side_length + 10), # Optional
     colors=None, # Optional
     isring=False,
     r=0.5,
     outputName = 'output/animation_pbc.mp4',
     PBC=True
 )

In [ ]:
# velocity correlation checking
velocity = np.load(velocity_file)   
#velocity

In [ ]:
# There're two functions that calculate time and spatial coorelation of velocity inside Cndbtools.py, using GPU acceleration
from chromdyn.traj_utils import Analyzer
bead_types = np.array(traj.ChromSeq)
v_correlation = Analyzer.calculate_vacf(
    velocities = velocity,
    bead_types = bead_types,
    sampling_step=1,
    platform='CPU'
)

v_spatial_correlation = Analyzer.calculate_spatial_vel_corr(
    coords = xyz,
    velocities = velocity,
    bead_types = bead_types,
    sampling_step=1,
    dist_range=10.0,
    num_bins=50,
    platform='CPU'
)

In [ ]:
# Each output is a dictionary with the correlation function for general and each bead type
# verify format if v_correlation
print(v_correlation.keys())
# plot the correlation function (time series)
v_plot = v_correlation['general']
import matplotlib.pyplot as plt

plt.figure()
plt.plot(v_plot)
plt.xlabel(r'Time lag(0.1 $\tau$)')
plt.ylabel('Velocity Autocorrelation')
plt.xlim(0,100)
plt.title('General Velocity Autocorrelation Function')
plt.show()



In [ ]:
# The output is a dictionary with the correlation function for general and each bead type
# verify format if v_correlation
print(v_spatial_correlation.keys())
# plot spatial correlation, x axis  = v_spatial_correlation['bin_centers'], y axis = ['general']
plt.figure(figsize=(10, 6))
plt.plot(v_spatial_correlation['bin_centers'], v_spatial_correlation['general'], label='General Correlation')
plt.xlabel('Distance')
plt.ylabel('Correlation')
plt.title('Spatial Correlation Function')
plt.legend()
plt.grid(True)
plt.show()
